In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
%matplotlib inline

In [4]:
# api_key = 'U93z8Z-2erhNJgpJ1mNy'
# api_url = "https://www.quandl.com/api/v3/datasets/WIKI/GOOGL/data.json?api_key={}".format(api_key)
# params = {'limit': 1000}
# response = requests.get(api_url, params=params)
# m_dict = response.json()
df = pd.read_csv('daily_TSLA.csv', parse_dates=['timestamp']).sort_values('timestamp').set_index('timestamp')
df.head()

,open,high,low,close,volume
timestamp,,,,,
2010-06-29,19.00,25.0000,17.54,23.89,18766300
2010-06-30,25.79,30.4192,23.30,23.83,17187100
2010-07-01,25.00,25.9200,20.27,21.96,8218800
2010-07-02,23.00,23.1000,18.71,19.20,5139800
2010-07-06,20.00,20.0000,15.83,16.11,6866900


In [5]:
small_ma = 50
long_ma = 200
sm = df.close.rolling(small_ma).mean()
lo = df.close.rolling(long_ma).mean()

In [7]:
def moving_average(df, small_ma, lomg_ma):
    sm = df.close.rolling(small_ma).mean()
    lo = df.close.rolling(long_ma).mean()
    buy = sm > lo
    sell = sm < lo
    return buy,sell

In [8]:
buy, sell = moving_average(df, 50, 200)

In [13]:
portfolio = 100
def backtest(df, buy, sell, capital, ini_shares, number = 10):
    portfolio = ini_shares
    cash = capital
    df['buy'] = buy
    df['sell'] = sell
    trades = []
    for index, row in df[(df['buy'] == True) | (df['sell'] == True)].iterrows():
        trade = {}
        trade['price'] = row['close']
        trade['timestamp'] = index
        if(row['buy']):
            portfolio += number
            cash = cash - number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'buy'
        elif(row['sell']):
            portfolio -= number
            cash = cash + number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'sell'
        trades.append(trade)
    trades_df = pd.DataFrame(trades).set_index('timestam')
    return trades_df.head()

In [14]:
backtest(df, buy, sell, 1000000, portfolio, 10)

ValueError: Shape of passed values is (5, 1808), indices imply (5, 1)